In [1]:
import os
import findspark

# Set the Spark installation path
os.environ["SPARK_HOME"] = "/home/elham/hadoop/spark-3.5.5"  
os.environ["PYSPARK_PYTHON"] = "python3"  

# Initialize findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
## To display notebook cell with horizontal scroll bar
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
# Replace with the path to your PostgreSQL JDBC jar file
jdbc_driver_path = "/home/elham/hadoop/spark-3.5.5/jars/postgresql-42.3.5.jar"

# starting spark session
spark = SparkSession.builder \
    .appName("Order Fact ETL") \
    .config("spark.jars", jdbc_driver_path) \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

25/04/13 15:41:46 WARN Utils: Your hostname, DESKTOP-CKKDTOM resolves to a loopback address: 127.0.1.1; using 172.30.19.133 instead (on interface eth0)
25/04/13 15:41:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/04/13 15:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 15:41:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
jdbc_url = "jdbc:postgresql://172.30.16.1:5432/olist_ecommerce"
properties = {
    "user": "postgres",
    "password": "31012009",
    "driver": "org.postgresql.Driver"
}

### Checking the schema and data types for the orders table and order items tables and viewing the data

In [6]:
order_table = spark.read.jdbc(url=jdbc_url, table="orders", properties=properties)
order_table.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [7]:
order_item_table = spark.read.jdbc(url=jdbc_url, table="order_items", properties=properties)
order_item_table.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [8]:
order_table.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:00|2017-10-02 11:07:00|         2017-10-04 19:55:00|          2017-10-10 21:25:00|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:00|2018-07-26 03:24:00|         2018-07-26 14:31:00|          2018-08-07 15:27:00|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [40]:
order_item_table.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:00|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:00| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:00| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:00| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:00| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

### Checking the amount of nulls and describing the data

In [10]:
# checking nulls in the order table
order_table.select([sum(col(c).isNull().cast("int")).alias(c+"_null") for c in order_table.columns]).show()

[Stage 2:>                                                          (0 + 1) / 1]

+-------------+----------------+-----------------+-----------------------------+----------------------+---------------------------------+----------------------------------+----------------------------------+
|order_id_null|customer_id_null|order_status_null|order_purchase_timestamp_null|order_approved_at_null|order_delivered_carrier_date_null|order_delivered_customer_date_null|order_estimated_delivery_date_null|
+-------------+----------------+-----------------+-----------------------------+----------------------+---------------------------------+----------------------------------+----------------------------------+
|            0|               0|                0|                            0|                   160|                             1783|                              2965|                                 0|
+-------------+----------------+-----------------+-----------------------------+----------------------+---------------------------------+-------------------------------

In [11]:
# checking nulls 
order_item_table.select([sum(col(c).isNull().cast("int")).alias(c+"_null") for c in order_item_table.columns]).show()

[Stage 5:>                                                          (0 + 1) / 1]

+-------------+------------------+---------------+--------------+------------------------+----------+------------------+
|order_id_null|order_item_id_null|product_id_null|seller_id_null|shipping_limit_date_null|price_null|freight_value_null|
+-------------+------------------+---------------+--------------+------------------------+----------+------------------+
|            0|                 0|              0|             0|                       0|         0|                 0|
+-------------+------------------+---------------+--------------+------------------------+----------+------------------+



In [12]:
order_table.describe().show()

[Stage 8:>                                                          (0 + 1) / 1]

+-------+--------------------+--------------------+------------+
|summary|            order_id|         customer_id|order_status|
+-------+--------------------+--------------------+------------+
|  count|               99441|               99441|       99441|
|   mean|                NULL|                NULL|        NULL|
| stddev|                NULL|                NULL|        NULL|
|    min|00010242fe8c5a6d1...|00012a2ce6f8dcda2...|    approved|
|    max|fffe41c64501cc87c...|ffffe8b65bbe3087b...| unavailable|
+-------+--------------------+--------------------+------------+



In [13]:
order_item_table.describe().show()

25/04/13 15:42:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 11:>                                                         (0 + 1) / 1]

+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+
|summary|            order_id|     order_item_id|          product_id|           seller_id|             price|     freight_value|
+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+
|  count|              112650|            112650|              112650|              112650|            112650|            112650|
|   mean|                NULL|1.1978339991122948|                NULL|                NULL|120.65373901477311| 19.99031992898562|
| stddev|                NULL|0.7051240313951734|                NULL|                NULL| 183.6339280502597|15.806405412296998|
|    min|00010242fe8c5a6d1...|                 1|00066f42aeeb9f300...|0015a82c2db000af6...|              0.85|               0.0|
|    max|fffe41c64501cc87c...|                21|fffe9eeff12fcbd74...|ffff564a4f9085cd2...

### Viewing the distinct order ids in both the orders table and the order_items table to see if there are any missing order ids and the reason behind it 

In [14]:
order_table.select("order_id").count()

99441

In [15]:
order_item_table.count()

112650

In [16]:
order_item_table.select("order_id").distinct().count()

98666

### Checking the values of the order status column

In [17]:
order_table.groupBy("order_status").count().show()

+------------+-----+
|order_status|count|
+------------+-----+
|    invoiced|  314|
|  processing|  301|
|     shipped| 1107|
| unavailable|  609|
|     created|    5|
|    approved|    2|
|   delivered|96478|
|    canceled|  625|
+------------+-----+



### Joining the orders table and the order_items table

In [18]:
order_df = order_table.join(
    order_item_table,
    on = "order_id",
    how = "left"
)

In [19]:
order_df.count()

113425

In [20]:
order_df.select([sum(col(c).isNull().cast("int")).alias(c+"_null") for c in order_df.columns]).show()

[Stage 42:=======================>                                  (2 + 3) / 5]

+-------------+----------------+-----------------+-----------------------------+----------------------+---------------------------------+----------------------------------+----------------------------------+------------------+---------------+--------------+------------------------+----------+------------------+
|order_id_null|customer_id_null|order_status_null|order_purchase_timestamp_null|order_approved_at_null|order_delivered_carrier_date_null|order_delivered_customer_date_null|order_estimated_delivery_date_null|order_item_id_null|product_id_null|seller_id_null|shipping_limit_date_null|price_null|freight_value_null|
+-------------+----------------+-----------------+-----------------------------+----------------------+---------------------------------+----------------------------------+----------------------------------+------------------+---------------+--------------+------------------------+----------+------------------+
|            0|               0|                0|           

In [21]:
order_df.select("order_id").count()

113425

In [22]:
order_df.select("order_id").distinct().count()

99441

In [23]:
order_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



### Extracting and dding the date key column(s) to match the date_sk in the date dimension

In [24]:
# extracting the date key from the date columns to match the date keys in the date dimension
order_df = order_df.withColumn(
    "purchase_date_key",
    date_format("order_purchase_timestamp", "yyyyMMdd").cast("int")
).withColumn(
    "order_approval_date_key",
    date_format("order_approved_at", "yyyyMMdd").cast("int")
).withColumn(
    "delivery_to_carrier_date_key",
    date_format("order_delivered_carrier_date", "yyyyMMdd").cast("int")
).withColumn(
    "delivery_to_customer_date_key",
    date_format("order_delivered_customer_date", "yyyyMMdd").cast("int")
).withColumn(
    "estimated_delivery_date_key",
    date_format("order_estimated_delivery_date", "yyyyMMdd").cast("int")
).withColumn(
    "shipping_limit_date_key",
    date_format("shipping_limit_date", "yyyyMMdd").cast("int")
)

In [25]:
order_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- purchase_date_key: integer (nullable = true)
 |-- order_approval_date_key: integer (nullable = true)
 |-- delivery_to_carrier_date_key: integer (nullable = true)
 |-- delivery_to_customer_date_key: integer (nullable = true)
 |-- estimated_delivery_date_key: integer (nullable = true)
 |-- shippin

### Calculating Fact measures and adding them to the order_df table

In [26]:
total_order_item_df = order_item_table.groupBy("order_id").agg(
    sum("price").alias("total_products_value"),
    sum("freight_value").alias("total_freight_value")
).withColumn(
    "total_order_price",
    (col("total_products_value") + col("total_freight_value")).cast("double")
)

In [27]:
order_df = order_df.join(
    total_order_item_df,
    on = "order_id",
    how = "left"
)

In [28]:
order_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+-----------------+-----------------------+----------------------------+-----------------------------+---------------------------+-----------------------+--------------------+-------------------+------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|purchase_date_key|order_approval_date_key|delivery_to_carrier_date_key|delivery_to_customer_date_key|estimated_delivery_date_key|shipping_limit_date_key|total_products_value|total_freight_value| total_order_price

### Connecting to the data warehouse and reading the dimensions

In [29]:
jdbc_url = "jdbc:postgresql://172.30.16.1:5432/olist_data_warehouse"

In [30]:
properties = {
    "user": "postgres",
    "password": "31012009",
    "driver": "org.postgresql.Driver"
}

In [31]:
customer_dim = spark.read.jdbc(url=jdbc_url, table="customer_dim", properties=properties)
seller_dim = spark.read.jdbc(url=jdbc_url, table="seller_dim", properties=properties)
product_dim = spark.read.jdbc(url=jdbc_url, table="product_dim", properties=properties)
payment_dim = spark.read.jdbc(url=jdbc_url, table="payment_dim", properties=properties)
reviews_dim = spark.read.jdbc(url=jdbc_url, table="reviews_dim", properties=properties)

### Joining the surrogate keys to the fact table

In [32]:
order_df = order_df \
.join(customer_dim.select("customer_id", "customer_sk"), on="customer_id", how="left") \
.join(seller_dim.select("seller_id", "seller_sk"), on="seller_id", how="left") \
.join(product_dim.select("product_id", "product_sk"), on="product_id", how="left") \
.join(payment_dim.select("order_id", "order_payment_sk"), on="order_id", how="left")\
.join(reviews_dim.select("order_id", "review_sk"), on="order_id", how="left")

In [33]:
order_df = order_df.withColumn("order_sk", monotonically_increasing_id())

In [34]:
order_df.select([sum(col(c).isNull().cast("int")).alias(c+"_null") for c in order_item_table.columns]).show()

[Stage 92:===========>                                              (1 + 4) / 5]

+-------------+------------------+---------------+--------------+------------------------+----------+------------------+
|order_id_null|order_item_id_null|product_id_null|seller_id_null|shipping_limit_date_null|price_null|freight_value_null|
+-------------+------------------+---------------+--------------+------------------------+----------+------------------+
|            0|               830|            830|           830|                     830|       830|               830|
+-------------+------------------+---------------+--------------+------------------------+----------+------------------+



In [35]:
order_df.count()

118762

In [36]:
order_df.show()

+--------------------+--------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+-------------------+------+-------------+-----------------+-----------------------+----------------------------+-----------------------------+---------------------------+-----------------------+--------------------+-------------------+------------------+-----------+---------+----------+----------------+---------+--------+
|            order_id|          product_id|           seller_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|shipping_limit_date| price|freight_value|purchase_date_key|order_approval_date_key|delivery_to_carrier_date_key|delivery_to_customer_date_key|estimated_delivery_date_key|shipping_limit

In [37]:
order_df.columns

['order_id',
 'product_id',
 'seller_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'shipping_limit_date',
 'price',
 'freight_value',
 'purchase_date_key',
 'order_approval_date_key',
 'delivery_to_carrier_date_key',
 'delivery_to_customer_date_key',
 'estimated_delivery_date_key',
 'shipping_limit_date_key',
 'total_products_value',
 'total_freight_value',
 'total_order_price',
 'customer_sk',
 'seller_sk',
 'product_sk',
 'order_payment_sk',
 'review_sk',
 'order_sk']

In [38]:
order_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- purchase_date_key: integer (nullable = true)
 |-- order_approval_date_key: integer (nullable = true)
 |-- delivery_to_carrier_date_key: integer (nullable = true)
 |-- delivery_to_customer_date_key: integer (nullable = true)
 |-- estimated_delivery_date_key: integer (nullable = true)
 |-- shippin

### Reordering Fact Columns and excluding business keys

In [42]:
order_fact = order_df.select(
    "order_sk",
    "customer_sk",
    "product_sk",
    "seller_sk",
    "review_sk",
    "order_payment_sk",
    "order_id",
    "order_item_id",
    "purchase_date_key",
    "order_approval_date_key",
    "delivery_to_carrier_date_key",
    "delivery_to_customer_date_key",
    "estimated_delivery_date_key",
    "shipping_limit_date_key",
    "order_status",
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "shipping_limit_date",
    "price",
    "freight_value",
    "total_products_value",
    "total_freight_value",
    "total_order_price"
)

In [43]:
order_fact.show()

+--------+-----------+----------+---------+---------+----------------+--------------------+-------------+-----------------+-----------------------+----------------------------+-----------------------------+---------------------------+-----------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-------------------+------+-------------+--------------------+-------------------+------------------+
|order_sk|customer_sk|product_sk|seller_sk|review_sk|order_payment_sk|            order_id|order_item_id|purchase_date_key|order_approval_date_key|delivery_to_carrier_date_key|delivery_to_customer_date_key|estimated_delivery_date_key|shipping_limit_date_key|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|shipping_limit_date| price|freight_value|total_products_value|total_freight_value| total_order_price|
+--------+-----------+----------+---------+---

In [45]:
order_fact.printSchema()

root
 |-- order_sk: long (nullable = false)
 |-- customer_sk: integer (nullable = true)
 |-- product_sk: integer (nullable = true)
 |-- seller_sk: integer (nullable = true)
 |-- review_sk: integer (nullable = true)
 |-- order_payment_sk: integer (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- purchase_date_key: integer (nullable = true)
 |-- order_approval_date_key: integer (nullable = true)
 |-- delivery_to_carrier_date_key: integer (nullable = true)
 |-- delivery_to_customer_date_key: integer (nullable = true)
 |-- estimated_delivery_date_key: integer (nullable = true)
 |-- shipping_limit_date_key: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- shipping_limit_dat

In [46]:
order_fact.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("output/order_fact.csv")